In [13]:
import copy
import requests
import re
import time
import pickle
import string
import nltk
import numpy as np
import pandas as pd
import json
import os


import keras
from random import choices
from keras.models import Sequential
from keras.layers import Conv1D, LeakyReLU
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.optimizers import SGD
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
os.chdir(os.path.join(os.getcwd(), 'drive', 'My Drive', 'Colab Notebooks'))
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Import, combine datagrip and online shopstyle data

In [20]:
local_data = pd.read_csv('result.csv')
online_data = pd.read_csv('online_data.csv')
train_data_2 = pd.read_csv('train_data_2.csv')

In [21]:
local_data = local_data[['Name', 'Desc', 'category_id']]
online_data = online_data[['Name', 'desc', 'Category']]

In [22]:
model_data = [local_data.rename(columns={"category_id":"Category"}), online_data.rename(columns={"desc":"Desc"})]
model_df = pd.concat(model_data, ignore_index=True)
frame = [model_df, train_data_2.rename(columns={'name':'Name', 'category':'Category', 'desc':'Desc'})]
model_df = pd.concat(frame, ignore_index=True)

# Sephora lip data cleaning and datagrip lip data

In [23]:
sephora = pd.read_csv('sephora_data_edited.csv')
other_data = pd.read_csv('other_data.csv')
face_data = pd.read_csv('face_data.csv')
other_data['Category'] = 'other'
other_data = other_data[['Name', 'Desc', 'Category']]
face_data = other_data[['Name', 'Desc', 'Category']]

In [24]:
sephora = sephora.dropna()
sephora = sephora.drop_duplicates()
other_data = other_data.dropna()
other_data = other_data.drop_duplicates()
face_data = face_data.dropna()
face_data = face_data.drop_duplicates()

In [25]:
frame = [sephora, model_df, other_data, face_data]
model_df = pd.concat(frame, ignore_index=True)

In [26]:
lip_data = pd.read_csv('lip_data.csv')
frame = [lip_data, model_df]
model_df = pd.concat(frame, ignore_index=True)

In [27]:
sephora_body = pd.read_csv('sephora_body_data_edited.csv')
sephora_body = sephora_body.dropna()
sephora_body = sephora_body.drop_duplicates()
sephora_body['Category'] = 'body'
model_df = pd.concat([model_df, sephora_body], ignore_index=True)

In [32]:
nail_index = model_df[model_df['Category']=='nail'].index
for index in nail_index:
  model_df.at[index, 'Category'] = 'hand'
print(model_df.shape)

(27684, 3)


In [ ]:
n = []
for i in model_df[model_df['Category']=='body'].index:
  print(i)
  desc = model_df.iloc[i]['Desc']
  print(desc)
  if desc.find('moist')!= -1 or desc.find('hydrat') != -1:
    n.append(i)
model_df.drop(index=n,inplace=True)

In [ ]:
for i in model_df.index:
  try:
    name = model_df.iloc[i]['Name'].lower()
    if name.find('reading') != -1:
      print(i)
      model_df.set_value(i, 'Category', 'other')
      print(model_df.iloc[i]['Name'])
    elif name.find('reader') != -1:
      print(name)
      model_df.set_value(i, 'Category', 'other')
      print(model_df.iloc[i]['Name'])
  except:
    continue

In [ ]:
for i in model_df.index:
  try:
    name = model_df.iloc[i]['Name'].lower()
    if name.find('facial') != -1:
      model_df.at[i, 'Category'] = 'face'
      print(model_df.iloc[i]['Name'])
    elif name.find('face') != -1:
      print(name)
      model_df.at[i, 'Category'] = 'face'
      print(model_df.iloc[i]['Name'])
  except:
    continue

# Add ready data

In [37]:
read_data = pd.read_csv('ready_data.csv')
print(read_data.shape)

(17429, 4)


In [38]:
ready_data = ready_data[['Category', 'Desc', 'Name']]
model_df = pd.concat([model_df, ready_data], ignore_index=True)

In [40]:
mapping = {'perfume': 'fragrance', 'toilette': 'fragrance', 'cologne': 'fragrance','parfum': 'fragrance','profumo': 'fragrance', 'fragrance': 'fragrance',
           'concealer': 'face', 'brush': 'face','face': 'face','bronzer': 'face','blush': 'face', 'essence': 'face', 'makeup': 'face', 'concentrate': 'face','concentrated': 'face',
           'lip balm': 'lip', 'lipstick': 'lip','lip': 'lip',
           'highlight': 'face','foundation': 'face','cheek': 'face', 'shaving': 'face', 'shave': 'face','facial': 'face', 'serum': 'face', 'acne': 'face',
           'anti-aging': 'face', 'day': 'face', 'night': 'face',
           'nail': 'hand', 'manicure': 'hand','polish': 'hand','hand': 'hand',
           'hair': 'hair', 'conditioner': 'hair','shampoo': 'hair',
           'eye': 'eye', 'brow': 'eye', 'lash': 'eye', 'eyeshadow': 'eye','mascara': 'eye', 'eyeliner': 'eye',
           'body': 'body', 'leg': 'body', 'bath':'body', 'shower': 'body', 'neck': 'body', 
           'eyeglass': 'other','reader': 'other', 'reading':'other', 'optical': 'other','eyeglasses': 'other',
           'frame': 'other', 'candle': 'other'
           }

In [41]:
new_mapping = {}
for k,v in mapping.items():
  if v in new_mapping.keys():
    new_mapping[v].append(k)
  else:
    new_mapping[v] = [k]
print(new_mapping)
alpha_list = ['other', 'eye','face','fragrance','hair','hand', 'lip', 'body']
mapping_list = []

for i in alpha_list:
  mapping_list.append({i:set(new_mapping[i])})
print(mapping_list)

{'fragrance': ['perfume', 'toilette', 'cologne', 'parfum', 'profumo', 'fragrance'], 'face': ['concealer', 'brush', 'face', 'bronzer', 'blush', 'essence', 'makeup', 'concentrate', 'concentrated', 'highlight', 'foundation', 'cheek', 'shaving', 'shave', 'facial', 'serum', 'acne', 'anti-aging', 'day', 'night'], 'lip': ['lip balm', 'lipstick', 'lip'], 'hand': ['nail', 'manicure', 'polish', 'hand'], 'hair': ['hair', 'conditioner', 'shampoo'], 'eye': ['eye', 'brow', 'lash', 'eyeshadow', 'mascara', 'eyeliner'], 'body': ['body', 'leg', 'bath', 'shower', 'neck'], 'other': ['eyeglass', 'reader', 'reading', 'optical', 'eyeglasses', 'frame', 'candle']}
[{'other': {'eyeglasses', 'eyeglass', 'reading', 'reader', 'optical', 'frame', 'candle'}}, {'eye': {'mascara', 'eyeliner', 'lash', 'eyeshadow', 'brow', 'eye'}}, {'face': {'blush', 'facial', 'bronzer', 'foundation', 'makeup', 'day', 'concentrate', 'essence', 'concentrated', 'cheek', 'anti-aging', 'acne', 'concealer', 'shave', 'night', 'face', 'shaving

In [43]:
for index in model_df.index:
  name_str_list = set(model_df.iloc[index]['Name'].lower().split(' '))
  for category in mapping_list:
    k, v = list(category.items())[0]
    if name_str_list.intersection(v):
      model_df.at[index, 'Category'] = k
      break
    else:
      description = set(model_df.iloc[index]['Desc'].lower().split(' '))
      if description.intersection(v):
        model_df.at[index, 'Category'] = k
        break

In [44]:
model_df.dropna(inplace=True)
model_df.drop_duplicates(inplace=True)
print(model_df.shape)

(20877, 3)


In [45]:
model_sample = model_df.sample(n = round(model_df.shape[0]*0.8))

In [53]:
# data samples
model_sample.sample(10)

,Name,Desc,Category
12853,Charlotte Tilbury Beauty Filter Pretty Youth G...,What It Is: Pretty Youth Glow Filter is a yout...,face
42746,"Confort Extreme Body Cream, 150ml - Colorless",Sisley's rich cream intensely nourishes and hy...,body
8152,Delectation Splendide perfume 100 ml,"An oriental, floral and spicy delight that rev...",fragrance
2501,"Estee Lauder Lip Duet Tint + Balm, Act Iv Coll...",What It Is: A mix-and-match lip duo with moist...,lip
9386,Jo Malone London Oud & Bergamot Cologne Intens...,Key Notes: - Top note: bergamot - Middle note:...,fragrance
31,MAC Tinted Lipglass,What It Is: MAC's classic Lipglass you adore i...,lip
23376,Creed White Amber 2.5 oz.,"Key Notes: Top notes: amber, vanilla, fruity a...",fragrance
14488,Nest Fragrances Dahlia & Vines Eau de Parfum 3...,Nest Fragrances Dahlia & Vines Eau de Parfum 3...,fragrance
22543,Skin Design London Alpine Rose Glow Illuminati...,What It Is: A super-hydrating daily antioxidan...,face
17531,Bobbi Brown Powder Puff,Use the Bobbi Brown Powder Puff to evenly appl...,face


In [ ]:
model_sample.to_csv('model_sample.csv')

# DNN model testing

In [98]:
model_df = pd.read_csv('model_sample.csv', encoding = "ISO-8859-1")
print(model_df.shape)

(16532, 3)


In [95]:
le = LabelEncoder()
le.fit(data_1['Category'])
y=le.transform(data_1['Category'])
n_class = len(list(le.classes_))
x = data_1['p_d'].tolist()

In [96]:
y

array([2, 6, 7, ..., 5, 0, 0])

In [97]:
max_word = 3000

tokenize = Tokenizer(num_words=max_word, lower=True, char_level=False)
tokenize.fit_on_texts(x)
x_train1 = tokenize.texts_to_matrix(x)


In [99]:
# build model
model = Sequential()
model.add(Dense(128, input_shape=(max_word,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#layer 1
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.35))
#layer 2
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
# # layer 3
# model.add(Dense(32))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(n_class))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
y_train1 = keras.utils.to_categorical(y, num_classes=len(np.unique(y)))
model.fit(x_train1, y_train1, batch_size=128, epochs=10, verbose=1, validation_split=0.1)
# score = model.evaluate(x_test1, y_test1, batch_size=32, verbose=1)
# print ('accuracy', score[1])
# model.save('DNN_1.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               384128    
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [ ]:
model.save('DNN_makeupmodel_2.h5')

# Model building

In [100]:
stemmer = SnowballStemmer(language='english')
stop_words = set(stopwords.words('english')) | set(string.punctuation)
def clean_train(text):
    """
    Lower-case, tokenize, stem,
    Removal: digits + punctuations + white spaces + stopwords
    """
    try:
      text = text.lower()
      raw = re.sub('[!\"#$%&()*+/:;<=,.>?@[\\]^_`{|}~]', ' ', text).split()
      t = ' '.join([stemmer.stem(w) for w in raw if w not in stop_words and not re.search('[\d]+',w)])
      return(t)
    except:
      return('')


In [101]:
data = model_df 
data_1 = copy.deepcopy(data) 
data_1 = data_1[data_1['Name'].notnull() & data_1['Desc'].notnull()]

data_1['p_d'] = data_1['Name']+' '+data_1['Desc']
data_1['p_d'] = data_1['p_d'].apply(clean_train)

In [102]:
n = model_df[model_df['Category'].isnull()].index
model_df.drop(index=n,inplace=True)

In [103]:
model_df.groupby('Category').count()

,Desc,Name
Category,,
body,1247,1247
eye,1884,1884
face,5314,5314
fragrance,4358,4358
hair,746,746
hand,547,547
lip,1674,1674
other,761,761


In [104]:
none_index = data_1[data_1['Category'].isnull()].index
data_1.drop(index=none_index, inplace=True)

In [105]:
## prepare data
le = LabelEncoder()
le.fit(data_1['Category'])
y=le.transform(data_1['Category'])
n_class = len(list(le.classes_))
x = data_1['p_d'].tolist()
## split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
max_word = 3000
tokenize = Tokenizer(num_words=max_word, lower=True, char_level=False)
tokenize.fit_on_texts(x_train)
x_train1 = tokenize.texts_to_matrix(x_train)
x_test1 = tokenize.texts_to_matrix(x_test)

In [ ]:
with open('encoder.pkl', 'wb') as f:
  pickle.dump(le, f)

In [ ]:
with open('tokenizer_unigram.pkl', 'wb') as handle:
  pickle.dump(tokenize, handle)

In [106]:
print(len(x_train), len(y_train), '\n', len(x_test), len(y_test))

13224 13224 
 3307 3307


# TFIDF model

In [157]:
def clean_train(text):
    """
    Lower-case, tokenize, stem,
    Removal: digits + punctuations + white spaces + stopwords
    """
    stemmer = SnowballStemmer(language='english')
    stop_words = set(stopwords.words('english')) | set(string.punctuation)
    try:
      text = text.lower()
      raw = re.sub('[!\"#$%&()*+/:;<=,.>?@[\\]^_`{|}~]', ' ', text).split()
      t = ' '.join([stemmer.stem(w) for w in raw if w not in stop_words and not re.search('[\d]+',w)])
      return(t)
    except:
      return('')

In [158]:
model_df = pd.read_csv('model_sample.csv', encoding = "ISO-8859-1")
data = model_df 
data_1 = copy.deepcopy(data) 
data_1 = data_1[data_1['Name'].notnull() & data_1['Desc'].notnull()]

data_1['p_d'] = data_1['Name']+' '+data_1['Desc']
data_1['p_d'] = data_1['p_d'].apply(clean_train)

In [159]:
with open('encoder.pkl', 'wb') as f:
  pickle.dump(le, f)

In [160]:
le = LabelEncoder()
le.fit(data_1['Category'])
y=le.transform(data_1['Category'])
n_class = len(list(le.classes_))
x = data_1['p_d'].tolist()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer


max_words=5000
tfidf_vectorizer = TfidfVectorizer(max_features=max_words, use_idf=True)
tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test =  tfidf_vectorizer.transform(x_test)
tf_train = tfidf_train.todense()
tf_test = tfidf_test.todense()
y_train1 = keras.utils.to_categorical(y_train, num_classes=len(np.unique(y)))
y_test1 = keras.utils.to_categorical(y_test, num_classes=len(np.unique(y)))
with open('vectorizer_tfidf_' + str(max_words) + '.pickle',
              'wb') as handle:
    pickle.dump(tfidf_vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
model2 = Sequential()
model2.add(Dense(64, input_shape=(max_words,)))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(64))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(n_class))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
model2.fit(tf_train, y_train1,
            batch_size=32,
            epochs=5,
            verbose=1,
            validation_split=0.1)
score = model2.evaluate(tf_test,y_test1, batch_size = 32, verbose=1)
print(score[1])

Epoch 1/5
372/372 [==============================] - 2s 4ms/step - loss: 1.0676 - accuracy: 0.6752 - val_loss: 0.5363 - val_accuracy: 0.8670
Epoch 2/5
372/372 [==============================] - 1s 4ms/step - loss: 0.4334 - accuracy: 0.8871 - val_loss: 0.3893 - val_accuracy: 0.8949
Epoch 3/5
372/372 [==============================] - 2s 4ms/step - loss: 0.2691 - accuracy: 0.9258 - val_loss: 0.3910 - val_accuracy: 0.8912
Epoch 4/5
372/372 [==============================] - 2s 4ms/step - loss: 0.1876 - accuracy: 0.9517 - val_loss: 0.4135 - val_accuracy: 0.8934
Epoch 5/5
104/104 [==============================] - 0s 2ms/step - loss: 0.3809 - accuracy: 0.9044
0.9044451117515564


In [ ]:
model2.save('tfidf_makeup_5000.h5')

In [163]:
model_test = pd.read_csv('beauty_test_2.csv')
print(model_test.shape)
model_test['n_d'] = model_test['name']+' '+model_test['info']
model_test['n_d'] = model_test['n_d'].apply(clean_train)
print(model_test['n_d'][0:5])
model_test_data =  tfidf_vectorizer.transform(model_test['n_d'])
print(model_test_data)

(600, 5)
0    derm institut cellular rejuven tighten serum r...
1    chantecaill anti-pollut finish essenc lightwei...
2    bobbi brown intens skin serum conceal porcelai...
3    oh naturell eyeshadow palett limited-edit eye ...
4    clarin eau des jardin eau de toilett uplift re...
Name: n_d, dtype: object
  (0, 4983)	0.10723324955618896
  (0, 4819)	0.09626497482739348
  (0, 4735)	0.18732546992255222
  (0, 4547)	0.14499379281045072
  (0, 4524)	0.15000165867413676
  (0, 4509)	0.5625283968141559
  (0, 4073)	0.15091326503271438
  (0, 3950)	0.19361467173270666
  (0, 3752)	0.11524004847088432
  (0, 3683)	0.16026581762348183
  (0, 3677)	0.2562093088795091
  (0, 3527)	0.15591550900923118
  (0, 3011)	0.10477629954604115
  (0, 2986)	0.12336006664774257
  (0, 2899)	0.12090552651248403
  (0, 2842)	0.12855271092859052
  (0, 2606)	0.10604455556364444
  (0, 2597)	0.14267459184234102
  (0, 2275)	0.17280464737992732
  (0, 2069)	0.22222193152454586
  (0, 1760)	0.09315043084386623
  (0, 1745)	0.1273391

In [164]:
model_test_data.sort_indices()
print(model_test_data)
gt_predict = model2.predict(model_test_data)

  (0, 197)	0.0890852303952447
  (0, 199)	0.0801299644962511
  (0, 723)	0.13394847836572668
  (0, 935)	0.10170158201413486
  (0, 1024)	0.0816604768854908
  (0, 1027)	0.14655553656043147
  (0, 1092)	0.11412929733416712
  (0, 1141)	0.18729770209325186
  (0, 1144)	0.18119279778898506
  (0, 1194)	0.1794462756218424
  (0, 1600)	0.08501039006458577
  (0, 1683)	0.10378004744502316
  (0, 1745)	0.1273391459360872
  (0, 1760)	0.09315043084386623
  (0, 2069)	0.22222193152454586
  (0, 2275)	0.17280464737992732
  (0, 2597)	0.14267459184234102
  (0, 2606)	0.10604455556364444
  (0, 2842)	0.12855271092859052
  (0, 2899)	0.12090552651248403
  (0, 2986)	0.12336006664774257
  (0, 3011)	0.10477629954604115
  (0, 3527)	0.15591550900923118
  (0, 3677)	0.2562093088795091
  (0, 3683)	0.16026581762348183
  :	:
  (599, 2270)	0.08602810791336352
  (599, 2352)	0.1806439592906862
  (599, 2404)	0.06780145109089446
  (599, 2527)	0.161014923652262
  (599, 2691)	0.13535864608192136
  (599, 2817)	0.08685874654613297
  (

In [ ]:
gt_predict = le.inverse_transform(np.argmax(gt_predict, axis=1))
print(gt_predict)
# model_test_data = pd.Series(gt_predict)
# model_test_data.to_csv('modelresult.csv')

# real random data test

In [69]:
beauty_test_v1 = pd.read_csv('beauty_test_2.csv')

In [ ]:
beauty_test_v1['n_d'] = beauty_test_v1['name']+' '+beauty_test_v1['info']
x_test = tokenize.texts_to_matrix(beauty_test_v1['n_d'])
y_predict = np.around(model.predict(x_test))
predict_t_test_data = le.inverse_transform(np.argmax(y_predict, axis=1))
beauty_test_v1['model_result'] = predict_t_test_data

In [ ]:
beauty_test_v1['model_result']

In [ ]:
beauty_test_v1.to_csv('beauty_test_with_result_2.csv',index=False)

# Model validation using datagrip data


In [71]:
from keras.models import load_model

In [72]:
model = load_model('DNN_4.h5')

In [73]:
with open('tokenizer_unigram.pkl', 'rb') as handle:
  tokenize = pickle.load(handle)
with open('encoder.pkl', 'rb') as f:
  encode = pickle.load(f)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 16)                48016     
_________________________________________________________________
activation_7 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                544       
_________________________________________________________________
activation_8 (Activation)    (None, 32)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 9)                

In [77]:
test_data = pd.read_csv('test_data_2.csv')
test_data.rename(columns={'Unnamed: 0':'category'}, inplace=True)

In [79]:
test_data['n_d'] = test_data['name']+ ' '+test_data['desc']
test_data['n_d'] = test_data['n_d'].apply(clean_train)

In [80]:
le = LabelEncoder()
le.fit(test_data['category'])

LabelEncoder()

In [81]:
x_test_data = tokenize.texts_to_matrix(test_data['n_d'])

In [82]:
y_test_data = np.around(model.predict(x_test_data))

In [ ]:
predict_t_test_data = le.inverse_transform(np.argmax(y_test_data, axis=1))

In [ ]:
test_data['model_result'] = predict_t_test_data
test_data['predict'] = [test_data['category'][i]==test_data['model_result'][i] for i in range(len(test_data))]
test_data[test_data['predict']==False].groupby('category').count()

In [ ]:
test_data=test_data.reset_index(drop=True)

In [ ]:
makeup_list = ['fragrance', 'eye', 'face', 'lip', 'hair', 'hand', 'body','other']
for makeup in makeup_list:
  print(makeup)
  test_data = model_df[model_df['Category']==makeup].sample(20)
  test_data['n_d'] = test_data['Name']+ ' '+test_data['Desc']
  test_data['n_d'] = test_data['n_d'].apply(clean_train)
  x_test_data = tokenize.texts_to_matrix(test_data['n_d'])
  y_test_data = np.around(model.predict(x_test_data))
  predict_t_test_data = le.inverse_transform(np.argmax(y_test_data, axis=1))
  test_data['model_result'] = predict_t_test_data
  test_data['predict'] = [test_data.iloc[i]['Category']==test_data.iloc[i]['model_result'] for i in range(len(test_data))]
  print(test_data[test_data['predict']==False].groupby('Category').count())

In [ ]:
test_data[test_data['predict']==False]

,Category,Desc,Name,n_d,model_result,predict
14501,other,Lightweight mist replenishes and energizes ski...,MOISTURE BOUND Skin Energy Hydration Delivery ...,moistur bound skin energi hydrat deliveri syst...,face,False
4588,other,Natura Bisse Diamond Instant Glow-Beauty & Cos...,Natura Bisse Diamond Instant Glow,natura biss diamond instant glow natura biss d...,face,False


In [ ]:
test_data.to_csv('test_data_hand.csv',index=False)

In [ ]:
y_test = keras.utils.to_categorical(test_data['category'], num_classes=len(np.unique(test_data['category'])))
model.evaluate(x_test_data, y_test, verbose=1)

In [ ]:
model_result = test_data[['category', 'name', 'desc', 'img_url', 'model_result', 'predict']]

In [ ]:
model_result.to_csv('model_result.csv')

In [67]:
from keras.models import load_model
with open('vectorizer_tfidf_5000.pickle', 'rb') as handle:
  vector = pickle.load(handle)
with open('encoder.pkl', 'rb') as f:
  encode = pickle.load(f)
model = load_model('tfidf_makeup_5000.h5')
def makeup_predict(name, desc, encoder = encode, tfvector = vector, model = model):
  """
  input:
      model, name of product, description of product
      model: a h5 file.
      name: a string.
      description: a string.
  return:
      category predicted by the model based on given name and description
  """

  try:
    n_d = name+' '+desc
  except TypeError:
    print('name and desc need to be string type.')
  n_d = [n_d]
  n_d = clean_train(n_d)
  result = model.predict(n_d)
  result = encode.inverse_transform(result)
  return result

In [ ]:
test = tokenize.texts_to_matrix(test)